In [1]:
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession

# SparkSession 생성
spark = (SparkSession
        .builder
        .appName("SparkSQLExampleApp")
        .getOrCreate())

In [5]:
import pandas as pd

In [7]:
df = pd.read_excel('../check_all_0719.xlsx')

In [8]:
df

,Unnamed: 0,Sales_date,Equip_num,Car_id,cluster_total,pred_total_L,Auction_entry,Model_name,Customer_price,Init_price,Expected_price,Sales_price,Auction_entry2,model_price,price_gap,price_gap_abs,MAPE
0,0,2021-05-10,10441808,KNAB3511BHT062288,1.0_GAS_A_기아_기아 모닝_Silver,40.413644,1,기아 모닝,13150000,4700000,4900000,5400000,first,5.314394e+06,-8.560581e+04,8.560581e+04,1.585293
1,1,2021-05-10,10443853,KNAB3511BHT080131,1.0_GAS_BCD_기아_기아 모닝_Silver,48.690413,1,기아 모닝,13150000,5600000,5900000,6550000,first,6.402789e+06,-1.472107e+05,1.472107e+05,2.247492
2,2,2021-05-10,10444240,KNAB6511BHT086734,1.0_GAS_BCD_기아_기아 모닝_Silver,42.812026,1,기아 모닝,11150000,4200000,4400000,4950000,first,4.773541e+06,-1.764591e+05,1.764591e+05,3.564829
3,3,2021-05-10,10444610,KNAB6511BHT077277,1.0_GAS_BCD_기아_기아 모닝_Silver,43.763981,1,기아 모닝,11150000,4300000,4500000,5350000,first,4.879684e+06,-4.703162e+05,4.703162e+05,8.790956
4,4,2021-05-10,10381624,KNABD511BGT240614,1.0_GAS_E_기아_기아 모닝_etc,35.352246,1,기아 모닝,10800000,3400000,3600000,3750000,first,3.818043e+06,6.804259e+04,6.804259e+04,1.814469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,3655,2021-07-12,10458138,KMHGL41DDJU253722,3.0_GAS_A_현대_현대 제네시스 G80_Black,53.943191,1,현대 제네시스 G80,51800000,26900000,27400000,27100000,first,2.794257e+07,8.425731e+05,8.425731e+05,3.109126
3656,3656,2021-07-12,10475011,KNALC411BJA120757,3.0_GAS_BCD_기아_기아 K7_Black,47.772267,1,기아 K7,41400000,18000000,18400000,22050000,first,1.977772e+07,-2.272281e+06,2.272281e+06,10.305130
3657,3657,2021-07-12,10448283,KNAME81ABJS371001,V2.2_DIE_A_기아_기아 카니발_Black,48.788552,1,기아 카니발,44190000,20000000,20500000,21750000,first,2.155966e+07,-1.903389e+05,1.903389e+05,0.875122
3658,3658,2021-07-12,10503790,KNAMC81ABKS495457,V2.2_DIE_BCD_기아_기아 카니발_Black,64.757646,1,기아 카니발,36400000,23100000,23600000,24350000,first,2.357178e+07,-7.782170e+05,7.782170e+05,3.195963


# 사용자 정의 함수

In [2]:
def cubed(s):
    return s*s*s

# UDF로 등록
spark.udf.register("cubed",cubed,LongType())

# 임시뷰 생성
spark.range(1,9).createOrReplaceTempView("udf_test")

In [3]:
spark.sql("SELECT * FROM udf_test").show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
+---+



In [4]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

Py4JJavaError: An error occurred while calling o40.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 16) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonf

## 판다스 UDF

In [8]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

def cubed(a:pd.Series) -> pd.Series:
    return a*a*a

cubed_udf = pandas_udf(cubed, returnType = LongType())

In [9]:
x = pd.Series([1,2,3])

print(cubed(x))

0     1
1     8
2    27
dtype: int64


### 스파크 데이터 프레임으로 전환

In [10]:
df = spark.range(1,4)

df.select("id", cubed_udf(col("id"))).show()

Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 48) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.sql.execution.python.ArrowEvalPythonExec.evaluate(ArrowEvalPythonExec.scala:88)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonf

# 고차함수

In [2]:
import pandas_gbq
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [3]:
from pyspark.sql.types import *
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

In [60]:
t_list = [[35,36,32,30,40,42,38]], [[31,32,34,55,56,0,0]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")

In [62]:
t_c.show()

Py4JJavaError: An error occurred while calling o241.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 42.0 failed 1 times, most recent failure: Lost task 0.0 in stage 42.0 (TID 92) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeO

## transform

In [10]:
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test").getOrCreate()
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

t_list = [[35,36,32,30,40,42,38]], [[31,32,34,55,56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")


In [11]:
t_c.show()

Py4JJavaError: An error occurred while calling o64.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea

In [15]:
spark.sql("SELECT celsius, transform(celsius, t -> (((t*9) div 5)+32)) AS fahrenheit FROM tC").show()

Py4JJavaError: An error occurred while calling o84.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea

## filter

In [17]:
spark.sql("""SELECT celsius,filter(celsius, t -> t > 38) AS high FROM tC """).show()

Py4JJavaError: An error occurred while calling o94.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea

## exists

In [23]:
spark.sql("SELECT celsius, exists(celsius, t -> t = 38) AS threshold FROM tC").show()

Py4JJavaError: An error occurred while calling o114.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (R14286N01.lotterental.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRe

## reduce

In [31]:
from functools import reduce
from pyspark.sql.functions import col

In [32]:
spark.sql("""SELECT celsius, reduce(celsius, 
                                  0, 
                                  (t,acc) -> t + acc, 
                                  acc -> (acc div size(celsius)*9 div 5) +32
                                  ) AS avgFahrenheit
            FROM tC""")

AnalysisException: Undefined function: reduce. This function is neither a built-in/temporary function, nor a persistent function that is qualified as spark_catalog.default.reduce.; line 1 pos 16

# 일반적인 데이터 프레임 및 스파크 SQL 작업

# p.152 - 관계형 연산

In [33]:
from pyspark.sql.functions import expr

tripdelaypath = 'C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv'
airportpath = 'C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt'


In [34]:
airportsna = (spark.read
             .format('csv')
             .options(header= 'true', inferSchema = "true", sep = '\t')
             .load(airportpath))
airportsna.createOrReplaceTempView("airports_na")

In [35]:
departureDelays = (spark.read
                  .format("csv")
                  .options(header = "true")
                  .load(tripdelaypath))

In [36]:
departureDelays = (departureDelays
                  .withColumn("delay", expr("CAST(delay as INT) as delay"))
                  .withColumn("distance", expr("CAST(distance as INT) as distance")))

departureDelays.createOrReplaceTempView("departureDelays")

In [37]:
# 임시 작은 테이블 생성

foo = (departureDelays
       .filter(expr("""origin == 'SEA' AND destination == 'SFO' and
       date like '01010%' and delay > 0""")))

foo.createOrReplaceTempView("foo")

In [38]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [39]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [40]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [41]:
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [42]:
# Union

bar = departureDelays.union(foo)
bar.createOrReplaceTempView("bar")

In [43]:
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO'
AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [44]:
#  Join
# 출발지연데이터(foo)와 공항 정보의 조인

foo.join(airportsna,
        airportsna.IATA == foo.origin).select("City", "State","date","delay","distance","destination").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



# 윈도우 함수

In [16]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")

DataFrame[]

In [18]:
departureDelays.show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [45]:
foo = (departureDelays
       .filter(expr("""origin == 'SEA' AND destination == 'SFO' and
       date like '01010%' and delay > 0""")))

foo.createOrReplaceTempView("foo")

In [47]:
window = spark.sql("""
SELECT origin, destination, SUM(delay) AS TotalDelays
FROM departureDelays
WHERE origin IN ("SEA","SFO","JFK")
AND destination IN ("SEA","SFO","JFK","DEN","ORD","LAX","ATL")
GROUP BY origin, destination
""")

In [50]:
window.createOrReplaceTempView("departureDelaysWindow")

In [51]:
spark.sql("SELECT * FROM departureDelaysWindow").show()

+------+-----------+-----------+
|origin|destination|TotalDelays|
+------+-----------+-----------+
|   JFK|        ORD|       5608|
|   JFK|        SFO|      35619|
|   JFK|        DEN|       4315|
|   JFK|        ATL|      12141|
|   JFK|        SEA|       7856|
|   JFK|        LAX|      35755|
|   SEA|        LAX|       9359|
|   SFO|        ORD|      27412|
|   SFO|        DEN|      18688|
|   SFO|        SEA|      17080|
|   SEA|        SFO|      22293|
|   SFO|        ATL|       5091|
|   SEA|        DEN|      13645|
|   SEA|        ATL|       4535|
|   SEA|        ORD|      10041|
|   SFO|        JFK|      24100|
|   SFO|        LAX|      40798|
|   SEA|        JFK|       4667|
+------+-----------+-----------+



In [52]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank
    FROM (
        SELECT origin, destination, TotalDelays,
        dense_rank() OVER (PARTITION BY origin ORDER BY TotalDelays DESC) AS rank
        FROM departureDelaysWindow
    )t
WHERE rank <=3
""").show()

+------+-----------+-----------+----+
|origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        ORD|      10041|   3|
|   SFO|        LAX|      40798|   1|
|   SFO|        ORD|      27412|   2|
|   SFO|        JFK|      24100|   3|
+------+-----------+-----------+----+



# 수정

In [21]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



## 열추가

In [22]:
from pyspark.sql.functions import expr

foo2 = (foo.withColumn(
        "status",
        expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
        ))

In [23]:
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



In [24]:
foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



## 칼럼명 바꾸기

In [25]:
foo4 = foo3.withColumnRenamed("status","flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



## 피벗

In [27]:
spark.sql("""SELECT destination, CAST(SUBSTRING(date,0,2) AS int) AS month, delay
    FROM departureDelays
    WHERE origin = 'SEA'
""").show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



In [29]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date,0,2) AS int) AS month, delay
    FROM departureDelays WHERE origin = 'SEA')
PIVOT (
    CAST(AVG(delay) AS DECIMAL(4,2)) AS AvgDelay, MAX(delay) AS MaxDelay
    FOR month IN (1 JAN, 2 FEB)
)
ORDER BY destination
""").show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D